# Supervised Learning
## classifiers
This project compares the outcome of the following models for two data sets:
- Decision Tree
- Neural Network
- kNN

## data sets
The data sets are:
1. BankMarketingData
2. PhishingWebsitesData

In [37]:
#importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import display # to display dataframe

dataPath = "../data/"

In [46]:

# loading Bank Marketing Data
def BankMarketingData():
    print(" Loading Bank Marketing Data... \n")
    df_bank = pd.read_csv(dataPath+"BankMarketingData.csv")

    print(f'{"number of samples":<22} | {"number of features":<22} | {"Any missing data":<22} \n{len(df_bank):<22} | {len(df_bank.columns):<22} | {Ture if df_bank.isnull().values.any() else False:<22}') 

    print(" \n\nsample bank data:")
    display(df_bank.head())


    print(" Preprocessing the data ...")
    


    #It should be noted that the feature 'pdays' is numeric but contains values that are '999' if the customer was not called before. It may be worth while to create a new feature that defines whether or not {0,1} a customer had been called before. In the current state the '999' values may be outliers. 
    #Finally we will standardize all numeric columns.    
    
    # 1- Some features have many categories, as listed below. Convert these features to one-hot encoding. 
    ToOneHot = ['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
    df_bank= pd.get_dummies(df_bank.astype(str), columns=ToOneHot)
    print("after dummies")
    display(df_bank.head())
    
    # 2- reordering
    column_order = list(df_bank)
    column_order.insert(0, column_order.pop(column_order.index('y')))
    df_bank = df_bank.loc[:, column_order]    
    print("after ordering")
    display(df_bank.head())    
    
    # 3- Convert the target variable from {no,yes} to {0,1} 
    df_bank['y'].replace("no","0",inplace=True)
    df_bank['y'].replace("yes","1",inplace=True) 
   
    df_bank['y'] = df_bank['y'].astype('category')    
    print("after replace ")
    display(df_bank.head())    
    
    # 4- Normalize numerical values
    numCols = ['age','duration','campaign','pdays','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']


    df_num = df_bank[numCols].astype(float)
    df_bank = df_bank.drop(numCols,axis=1)
    df_bank=df_bank.join( (df_num-df_num.min())/(df_num.max()-df_num.min()) )
    df_bank.describe(include='all')    
    print("final ")
    display(df_bank.head())    

    
    return df_bank

BankMarketingData()

 Loading Bank Marketing Data... 

number of samples      | number of features     | Any missing data       
45307                  | 21                     | 0                     
 

sample bank data:


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


 Preprocessing the data ...
after dummies


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0


after ordering


...

after replace 


,y,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,56,261,1,999,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,0,0,0,0,1,0
1,0,57,149,1,999,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,0,0,0,0,1,0
2,0,37,226,1,999,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,0,0,0,0,1,0
3,0,40,151,1,999,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,0,0,0,0,1,0
4,0,56,307,1,999,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,0,0,0,0,1,0


final 


,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0,0,0,0,1,0,0,0,0,0,...,0.481481,0.053070,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735
1,0,0,0,0,0,0,0,0,1,0,...,0.493827,0.030297,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735
2,0,0,0,0,0,0,0,0,1,0,...,0.246914,0.045954,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735
3,0,1,0,0,0,0,0,0,0,0,...,0.283951,0.030704,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735
4,0,0,0,0,0,0,0,0,1,0,...,0.481481,0.062424,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735


,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0,0,0,0,1,0,0,0,0,0,...,0.481481,0.053070,0.000000,1.0,0.000000,0.937500,0.698753,0.602510,0.957379,0.859735
1,0,0,0,0,0,0,0,0,1,0,...,0.493827,0.030297,0.000000,1.0,0.000000,0.937500,0.698753,0.602510,0.957379,0.859735
2,0,0,0,0,0,0,0,0,1,0,...,0.246914,0.045954,0.000000,1.0,0.000000,0.937500,0.698753,0.602510,0.957379,0.859735
3,0,1,0,0,0,0,0,0,0,0,...,0.283951,0.030704,0.000000,1.0,0.000000,0.937500,0.698753,0.602510,0.957379,0.859735
4,0,0,0,0,0,0,0,0,1,0,...,0.481481,0.062424,0.000000,1.0,0.000000,0.937500,0.698753,0.602510,0.957379,0.859735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,0,1,0,0,0,0,0,0,0,0,...,0.160494,0.010777,0.000000,1.0,0.000000,1.000000,0.669135,0.338912,0.980277,1.000000
45303,0,1,0,0,0,0,0,0,0,0,...,0.271605,0.044530,0.000000,1.0,0.000000,1.000000,0.669135,0.338912,0.980503,1.000000
45304,0,0,0,0,0,0,0,0,0,1,...,0.123457,0.013013,0.018182,1.0,0.142857,0.333333,0.269680,0.192469,0.163228,0.512287
45305,0,1,0,0,0,0,0,0,0,0,...,0.506173,0.107361,0.000000,1.0,0.000000,1.000000,0.484412,0.615063,0.982090,1.000000


In [39]:

# loading and processing Phishing Websites Data
def PhishingWebsitesData():
    print(" Loading Phishing Websites Data")
    df_phish = pd.read_csv(dataPath+"PhishingWebsitesData.csv")

    print(f'{"number of samples":<22} | {"number of features":<22} | {"Any missing data":<22} \n{len(df_phish):<22} | {len(df_phish.columns):<22} | {Ture if df_phish.isnull().values.any() else False:<22}') 

    print(" \n\nsample phishing data:")
    display(df_phish.head())

    print(" Cleansing/Preprocessing the data ...")

    # 1- Some features are {-1,0,1}. Convert these features to one-hot encoding. 
    ThreeCatFeatures = ['URL_Length','having_Sub_Domain','SSLfinal_State','URL_of_Anchor','Links_in_tags','SFH','web_traffic','Links_pointing_to_page']
    df_phish= pd.get_dummies(df_phish.astype(str), columns=ThreeCatFeatures)
    print("after dummies")
    display(df_phish.head())
    
    
    # 2- The rest of the features of (except redirect) has -1,1 values. Converting those to 0/1, by replacing -1 to 0.    
    df_phish = df_phish.replace('-1','0') # ,inplace=True
    print("after replace")
    display(df_phish.head())
    
    # 3- moving the results to the first column
    column_order = list(df_phish)
    column_order.insert(0, column_order.pop(column_order.index('Result')))
    df_phish = df_phish.loc[:, column_order]   
    df_phish.describe(include='all')
    display(df_phish.head())
    
    return df_phish

PhishingWebsitesData()


 Loading Phishing Websites Data
number of samples      | number of features     | Any missing data       
11055                  | 31                     | 0                     
 

sample phishing data:


,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


 Cleansing/Preprocessing the data ...
after dummies


,having_IP_Address,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,...,Links_in_tags_1,SFH_-1,SFH_0,SFH_1,web_traffic_-1,web_traffic_0,web_traffic_1,Links_pointing_to_page_-1,Links_pointing_to_page_0,Links_pointing_to_page_1
0,-1,1,1,-1,-1,-1,1,1,-1,1,...,1,1,0,0,1,0,0,0,0,1
1,1,1,1,1,-1,-1,1,1,-1,1,...,0,1,0,0,0,1,0,0,0,1
2,1,1,1,1,-1,-1,1,1,-1,1,...,0,1,0,0,0,0,1,0,1,0
3,1,1,1,1,-1,1,1,1,-1,-1,...,0,1,0,0,0,0,1,1,0,0
4,1,-1,1,1,-1,-1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,1


after replace


,having_IP_Address,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,...,Links_in_tags_1,SFH_-1,SFH_0,SFH_1,web_traffic_-1,web_traffic_0,web_traffic_1,Links_pointing_to_page_-1,Links_pointing_to_page_0,Links_pointing_to_page_1
0,0,1,1,0,0,0,1,1,0,1,...,1,1,0,0,1,0,0,0,0,1
1,1,1,1,1,0,0,1,1,0,1,...,0,1,0,0,0,1,0,0,0,1
2,1,1,1,1,0,0,1,1,0,1,...,0,1,0,0,0,0,1,0,1,0
3,1,1,1,1,0,1,1,1,0,0,...,0,1,0,0,0,0,1,1,0,0
4,1,0,1,1,0,0,1,1,1,1,...,0,1,0,0,0,1,0,0,0,1


,Result,having_IP_Address,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,Domain_registeration_length,Favicon,port,HTTPS_token,...,Links_in_tags_1,SFH_-1,SFH_0,SFH_1,web_traffic_-1,web_traffic_0,web_traffic_1,Links_pointing_to_page_-1,Links_pointing_to_page_0,Links_pointing_to_page_1
0,0,0,1,1,0,0,0,1,1,0,...,1,1,0,0,1,0,0,0,0,1
1,0,1,1,1,1,0,0,1,1,0,...,0,1,0,0,0,1,0,0,0,1
2,0,1,1,1,1,0,0,1,1,0,...,0,1,0,0,0,0,1,0,1,0
3,0,1,1,1,1,0,1,1,1,0,...,0,1,0,0,0,0,1,1,0,0
4,1,1,0,1,1,0,0,1,1,1,...,0,1,0,0,0,1,0,0,0,1


,Result,having_IP_Address,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,Domain_registeration_length,Favicon,port,HTTPS_token,...,Links_in_tags_1,SFH_-1,SFH_0,SFH_1,web_traffic_-1,web_traffic_0,web_traffic_1,Links_pointing_to_page_-1,Links_pointing_to_page_0,Links_pointing_to_page_1
0,0,0,1,1,0,0,0,1,1,0,...,1,1,0,0,1,0,0,0,0,1
1,0,1,1,1,1,0,0,1,1,0,...,0,1,0,0,0,1,0,0,0,1
2,0,1,1,1,1,0,0,1,1,0,...,0,1,0,0,0,0,1,0,1,0
3,0,1,1,1,1,0,1,1,1,0,...,0,1,0,0,0,0,1,1,0,0
4,1,1,0,1,1,0,0,1,1,1,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,1,1,1,0,1,1,0,0,0,1,...,1,1,0,0,1,0,0,0,0,1
11051,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,1,0,0
11052,0,1,1,1,1,0,0,1,1,1,...,0,1,0,0,0,0,1,0,1,0
11053,0,0,1,1,1,0,1,0,1,1,...,1,1,0,0,0,0,1,0,0,1


In [54]:
# loading and processing adult Data

# info about the data:

# income >50K, <=50K.
# age: continuous.
# workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
# fnlwgt: continuous.
# education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
# education-num: continuous.
# marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
# occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
# relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
# race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
# sex: Female, Male.
# capital-gain: continuous.
# capital-loss: continuous.
# hours-per-week: continuous.
# native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

def adult():
    print(" Loading adult Data")
    df_adult = pd.read_csv(dataPath+"adult/adult.data", header=None, na_values=" ?")

    print(f'{"number of samples":<22} | {"number of features":<22} | {"Any missing data":<22} \n{len(df_adult):<22} | {len(df_adult.columns):<22} | {True if df_adult.isnull().values.any() else False:<22}') 

    print(" \n\nsample adult data:")
    df_adult.head()

    print(" Cleansing/Preprocessing the data ...")
    # 1- defining the columns/feature names
    #df_adult.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
    df_adult.columns = ['age','employer',   'fnlwt',  'edu',       'edu_num',       'marital',        'occupation','relationship','race','sex','cap_gain','cap_loss','hrs','country','income']
    
    #print(df_adult.columns)
    #print(df_adult['native-country'])
   
    # 2- merging cap features and dropping unnecessary vars: cap_gain > 0 => cap_loss = 0 and vice versa.
    df_adult['cap_gain_loss'] = df_adult['cap_gain']-df_adult['cap_loss']
    df_adult = df_adult.drop(['fnlwt','edu','cap_gain','cap_loss'],1)
    
    # 3- one-hot encoding of income
    df_adult['income'] = pd.get_dummies(df_adult.income)

    
    # 4- replace cats
    #http://scg.sdsu.edu/dataset-adult_r/
    replacements = { 'Cambodia':' SE-Asia',
                    'Canada':' British-Commonwealth',
                    'China':' China',
                    'Columbia':' South-America',
                    'Cuba':' Other',
                    'Dominican-Republic':' Latin-America',
                    'Ecuador':' South-America',
                    'El-Salvador':' South-America ',
                    'England':' British-Commonwealth',
                    'France':' Euro_1',
                    'Germany':' Euro_1',
                    'Greece':' Euro_2',
                    'Guatemala':' Latin-America',
                    'Haiti':' Latin-America',
                    'Holand-Netherlands':' Euro_1',
                    'Honduras':' Latin-America',
                    'Hong':' China',
                    'Hungary':' Euro_2',
                    'India':' British-Commonwealth',
                    'Iran':' Other',
                    'Ireland':' British-Commonwealth',
                    'Italy':' Euro_1',
                    'Jamaica':' Latin-America',
                    'Japan':' Other',
                    'Laos':' SE-Asia',
                    'Mexico':' Latin-America',
                    'Nicaragua':' Latin-America',
                    'Outlying-US(Guam-USVI-etc)':' Latin-America',
                    'Peru':' South-America',
                    'Philippines':' SE-Asia',
                    'Poland':' Euro_2',
                    'Portugal':' Euro_2',
                    'Puerto-Rico':' Latin-America',
                    'Scotland':' British-Commonwealth',
                    'South':' Euro_2',
                    'Taiwan':' China',
                    'Thailand':' SE-Asia',
                    'Trinadad&Tobago':' Latin-America',
                    'United-States':' United-States',
                    'Vietnam':' SE-Asia',
                    'Yugoslavia':' Euro_2'}
    df_adult['country'] = df_adult['country'].str.strip()
    df_adult = df_adult.replace(to_replace={'country':replacements,
                                      'employer':{' Without-pay': ' Never-worked'},
                                      'relationship':{' Husband': 'Spouse',' Wife':'Spouse'}})        
    df_adult['country'] = df_adult['country'].str.strip()
    
    
    # 5- cleaning
    for col in ['employer','marital','occupation','relationship','race','sex','country']:
        df_adult[col] = df_adult[col].str.strip()
    
    # one-hot encoding adult
    df_adult = pd.get_dummies(df_adult)
    df_adult = df_adult.rename(columns=lambda x: x.replace('-','_'))

    print("final adult: ")
    display(df_adult.tail())

    
    #======
    madX1 = pd.read_csv(dataPath+'madelon_train.data',header=None,sep=' ')
    madX2 = pd.read_csv(dataPath+'madelon_valid.data',header=None,sep=' ')
    madX = pd.concat([madX1,madX2],0).astype(float)
    madY1 = pd.read_csv(dataPath+'madelon_train.labels',header=None,sep=' ')
    madY2 = pd.read_csv(dataPath+'madelon_valid.labels',header=None,sep=' ')
    madY = pd.concat([madY1,madY2],0)
    madY.columns = ['Class']
    mad = pd.concat([madX,madY],1)
    mad = mad.dropna(axis=1,how='all')

    print("final adult: ")
    display(mad.tail())
    
    
    return [df_adult, mad]

adult()






 Loading adult Data
number of samples      | number of features     | Any missing data       
32561                  | 15                     | 1                     
 

sample adult data:
 Cleansing/Preprocessing the data ...
final adult: 


,age,edu_num,hrs,income,cap_gain_loss,employer_Federal_gov,employer_Local_gov,employer_Never_worked,employer_Private,employer_Self_emp_inc,...,sex_Male,country_British_Commonwealth,country_China,country_Euro_1,country_Euro_2,country_Latin_America,country_Other,country_SE_Asia,country_South_America,country_United_States
32556,27,12,38,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
32557,40,9,40,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
32558,58,9,40,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
32559,22,9,20,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
32560,52,9,40,0,15024,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


final adult: 


,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,Class
595,493.0,458.0,503.0,478.0,517.0,479.0,472.0,478.0,444.0,477.0,...,475.0,485.0,443.0,517.0,486.0,474.0,489.0,506.0,506.0,1
596,481.0,484.0,481.0,490.0,449.0,481.0,467.0,478.0,469.0,483.0,...,485.0,508.0,599.0,498.0,527.0,481.0,490.0,455.0,451.0,1
597,485.0,485.0,530.0,480.0,444.0,487.0,462.0,475.0,509.0,494.0,...,474.0,502.0,368.0,453.0,482.0,478.0,481.0,484.0,517.0,1
598,477.0,469.0,528.0,485.0,483.0,469.0,482.0,477.0,494.0,476.0,...,476.0,453.0,638.0,471.0,538.0,470.0,490.0,613.0,492.0,1
599,482.0,453.0,515.0,481.0,500.0,493.0,503.0,477.0,501.0,475.0,...,478.0,487.0,694.0,493.0,499.0,474.0,494.0,536.0,526.0,-1


[       age  edu_num  hrs  income  cap_gain_loss  employer_Federal_gov  \
 0       39       13   40       1           2174                     0   
 1       50       13   13       1              0                     0   
 2       38        9   40       1              0                     0   
 3       53        7   40       1              0                     0   
 4       28       13   40       1              0                     0   
 ...    ...      ...  ...     ...            ...                   ...   
 32556   27       12   38       1              0                     0   
 32557   40        9   40       0              0                     0   
 32558   58        9   40       1              0                     0   
 32559   22        9   20       1              0                     0   
 32560   52        9   40       0          15024                     0   
 
        employer_Local_gov  employer_Never_worked  employer_Private  \
 0                       0             